In [142]:
import itertools
import numpy as np

In [2]:
example = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""

In [3]:
input = example.splitlines()

In [4]:
with open("input.txt") as f:
    input = f.read().splitlines()

In [8]:
instruct = input[0]
map_dict = {entry[0:3] : (entry[7:10], entry[12:15]) for entry in input[2:]}

steps = 0
key = 'AAA'

for c in itertools.cycle(instruct):
    steps += 1
    if c == 'L':
        key = map_dict[key][0]
    elif c == 'R':
        key = map_dict[key][1]
    else:
        break

    if key == 'ZZZ' or steps > 1e6:
        break

print(steps)
print(key)

12083
ZZZ


## Part 2

In [154]:
example = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)"""

input = example.splitlines()

In [155]:
instruct = input[0]
map_dict = {entry[0:3] : (entry[7:10], entry[12:15]) for entry in input[2:]}
keys_in = [key for key in map_dict.keys() if key[-1] == 'A']
keys_out = [key for key in map_dict.keys() if key[-1] == 'Z']

In [156]:
print(keys_in, keys_out)

['11A', '22A'] ['11Z', '22Z']


In [157]:
def get_min_steps(key_in, key_out, instructions, map_dict, step_limit = 1e5):
    key = key_in
    steps = 0

    for c in itertools.cycle(instruct):
        steps += 1

        if steps > step_limit:
            raise Exception("Exceeded step limit")
        
        d = 0 if c == 'L' else 1
        key = map_dict[key][d]

        if key == key_out:
            break
    
    return steps

In [158]:
in_steps = []
out_steps = []

for (k_in, k_out) in itertools.product(keys_in, keys_out):
    try:
        in_steps.append([k_in, k_out, get_min_steps(k_in, k_out, instruct, map_dict)])
    except:
        pass

for (k_in, k_out) in itertools.product(keys_out, keys_out):
    try:
        out_steps.append([k_in, k_out, get_min_steps(k_in, k_out, instruct, map_dict)])
    except:
        pass

print(in_steps, "\n", out_steps)

[['11A', '11Z', 2], ['22A', '22Z', 3]] 
 [['11Z', '11Z', 2], ['22Z', '22Z', 3]]


In [98]:
with open("input.txt") as f:
    input = f.read().splitlines()

In [99]:
instruct = input[0]
map_dict = {entry[0:3] : (entry[7:10], entry[12:15]) for entry in input[2:]}
keys_in = [key for key in map_dict.keys() if key[-1] == 'A']
keys_out = [key for key in map_dict.keys() if key[-1] == 'Z']

In [100]:
print(keys_in, "\n", keys_out)

['FSA', 'JVA', 'QXA', 'KNA', 'AAA', 'FXA'] 
 ['CJZ', 'ZZZ', 'SLZ', 'QCZ', 'PFZ', 'GNZ']


In [145]:
in_steps = []

for (k_in, k_out) in itertools.product(keys_in, keys_out):
    try:
        in_steps.append([k_in, k_out, get_min_steps(k_in, k_out, instruct, map_dict)])
    except:
        pass

In [128]:
print(in_steps)

[['FSA', 'QCZ', 20513], ['JVA', 'CJZ', 18827], ['QXA', 'PFZ', 17141], ['KNA', 'GNZ', 22199], ['AAA', 'ZZZ', 12083], ['FXA', 'SLZ', 13207]]


In [129]:
[i[-1] % len(instruct) for i in in_steps]

[0, 0, 0, 0, 0, 0]

In [135]:
out_steps = []

for (k_in, k_out) in itertools.product(keys_out, keys_out):
    try:
        out_steps.append([k_in, k_out, get_min_steps(k_in, k_out, instruct, map_dict)])
    except:
        pass

print(out_steps)

[['CJZ', 'CJZ', 18827], ['ZZZ', 'ZZZ', 12083], ['SLZ', 'SLZ', 13207], ['QCZ', 'QCZ', 20513], ['PFZ', 'PFZ', 17141], ['GNZ', 'GNZ', 22199]]


In [132]:
[i[-1] % len(instruct) for i in out_steps]

[0, 0, 0, 0, 0, 0]

So, this problem is made MUCH easier by the fact that there's really only one instruction, since each of the xxA's are connected to the xxZ's by an integer multiple of instruct and each of the xxZ's are connected to themselves by an integer multiple of instruct 

In [146]:
out_steps = {}

for k_in in keys_out:
    try:
        out_steps[k_in] = get_min_steps(k_in, k_in, instruct, map_dict)
    except:
        pass

print(out_steps)

{'CJZ': 18827, 'ZZZ': 12083, 'SLZ': 13207, 'QCZ': 20513, 'PFZ': 17141, 'GNZ': 22199}


In [147]:
cycles = in_steps

for i in cycles:
    i.append(out_steps[i[1]])

cycles

[['FSA', 'QCZ', 20513, 20513],
 ['JVA', 'CJZ', 18827, 18827],
 ['QXA', 'PFZ', 17141, 17141],
 ['KNA', 'GNZ', 22199, 22199],
 ['AAA', 'ZZZ', 12083, 12083],
 ['FXA', 'SLZ', 13207, 13207]]

Also made much easier by the fact that each subsequent cycle is the same length as the initial cycle needed. So we just want the lowest common multiple

In [150]:
np.lcm.reduce([i[-1] for i in cycles])

13385272668829